In [1]:
import pandas as pd
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tqdm import tqdm 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Import des données

In [2]:
apple = pd.read_csv("../data/AAPL.csv")
amazon = pd.read_csv("../data/AMZN.csv")
berkshire = pd.read_csv("../data/BRK-B.csv")
google = pd.read_csv("../data/GOOG.csv")
meta = pd.read_csv("../data/META.csv")
microsoft = pd.read_csv("../data/MSFT.csv")
nvidia = pd.read_csv("../data/NVDA.csv")
tesla = pd.read_csv("../data/TSLA.csv")
unitedhealth = pd.read_csv("../data/UNH.csv")
exxon = pd.read_csv("../data/XOM.csv")

In [3]:
titres = {"Apple" : apple, 
          "Amazon" : amazon,
          "Berkshire" : berkshire,
          "Google" : google,
          "Meta" : meta,
          "Microsoft" : microsoft,
          "Nvidia" : nvidia,
          "Tesla" : tesla,
          "UnitedHealth" : unitedhealth,
          "Exxon" : exxon}

# Calcul des prédictions

In [4]:
def stock_volatility_forecast(stock):
    stock['Date'] = pd.to_datetime(stock['Date'])
    stock['Volatility_HighLow_10days'] = stock['High'].rolling(window=10).std() / stock['Low'].rolling(window=10).mean()
    stock['Return'] = stock['Close'].pct_change()
    stock = stock.dropna()
    stock = stock.drop(columns=["Open", 'Adj Close'])

    # Calcul de la volatilité sur une fenêtre mobile
    stock['Volatility'] = stock['Return'].rolling(window=10).std()

    # Définition de la cible comme la volatilité future
    stock['Target'] = stock['Volatility'].shift(-7)
    stock = stock.dropna()

    window_size = 90
    step_size = 7
    prediction_days = 7

    prediction_dates = []
    predictions = []
    for start in tqdm(range(0, len(stock) - window_size - prediction_days, step_size)):
        end = start + window_size
        train = stock.iloc[start:end]
        test = stock.iloc[end:end + prediction_days]

        model = XGBRegressor()
        model.fit(train.drop(['Date', 'Target', 'Volume'], axis=1), train['Target'])

        pred = model.predict(test.drop(['Date', 'Target', 'Volume'], axis=1))
        predictions.append(pred)

        prediction_date = stock.iloc[end + prediction_days - 1]['Date']
        prediction_dates.append(prediction_date)

    predictions = [value[6] for value in predictions]

    df_predictions = pd.DataFrame({
        'Date': prediction_dates,
        'Prediction': predictions
    })

    merged_df = stock.merge(df_predictions, on='Date', how='inner')

    merged_df = merged_df.drop(columns = ["Target"])

    return(merged_df)


In [8]:
predictions_dic = {}

In [9]:
for key in titres :
    predictions_dic[key] = stock_volatility_forecast(titres[key])

100%|██████████| 378/378 [00:10<00:00, 36.20it/s]


In [12]:
dfs_to_concat = []

for name, df in predictions_dic.items():
    df['Date'] = pd.to_datetime(df['Date'])
    dfs_to_concat.append(df[['Date', 'Prediction', "Volatility"]])

portfolio_predictions = pd.concat(dfs_to_concat)
portfolio_avg_predictions = portfolio_predictions.groupby('Date').mean().reset_index()

print(portfolio_avg_predictions.head())

        Date  Prediction  Volatility
0 2013-06-17    0.017316    0.014440
1 2013-06-26    0.014908    0.014275
2 2013-07-08    0.017694    0.012786
3 2013-07-17    0.015491    0.014141
4 2013-07-26    0.017408    0.027492


In [15]:
fig = go.Figure()

# Ajout de la série financière
fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Prediction'],
                         mode = 'lines',
                         name ='Prédiction', 
                         fill = "tozeroy"))

fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Volatility'],
                         mode = 'lines',
                         name ='Réel', 
                         fill = "tozeroy"))

# Mise à jour des titres
fig.update_layout(title="Évolution de la volatilité du ptf",
                  xaxis_title='Date',
                  yaxis_title='Valeur')

# Affichage du graphique
fig.show()